# ItaData2024

## Pneumonia

propositional analysis

In [28]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Settings

In [29]:
experiment = :Pneumonia
# experiment = :Bronchiectasis
# experiment = :COPD
# experiment = :URTI
# experiment = :Bronchiolitis

features = :catch9
# features = :minmax
# features = :custom

loadset = false
# loadset = true

scale = :semitones
# scale = :mel_htk

usemfcc = false
# usemfcc = true
usef0 = false
# usef0 = true

sr = 8000
audioparams = (
    usemfcc = usemfcc,
    usef0 = usef0,
    sr = sr,
    nfft = 512,
    scale = scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    nbands = scale == :semitones ? 14 : 26,
    ncoeffs = scale == :semitones ? 7 : 13,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = usemfcc ? false : true,
)

memguard = false;
# memguard = true;
n_elems = 15;

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

@assert experiment in avail_exp "Unknown type of experiment: $experiment."

findhealthy = y -> findall(x -> x == "Healthy", y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment)
findsick = y -> findall(x -> x == String(experiment), y)
filename = "/datasets/itadata2024_" * String(experiment) * "_files"
memguard && begin filename *= string("_memguard") end

destpath = "results/propositional"
jld2file = destpath * "/itadata2024_" * String(experiment) * "_" * String(scale) * ".jld2"

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

### Audio features extraction function
This function is called for every audio sample and extract 25 features:
14 bands of the mel spectrogram,
11 spectral features: centroid, crest, entropy, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [30]:
function afe(x::AbstractVector{Float64}, audioparams::NamedTuple; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = audioparams.sr
    norm = true
    speech_detection = false
    # stft module
    nfft = audioparams.nfft
    win_type = (:hann, :periodic)
    win_length = audioparams.nfft
    overlap_length = round(Int, audioparams.nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = audioparams.nbands
    scale = audioparams.scale               # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = audioparams.freq_range
    # mel spectrogram module
    db_scale = audioparams.db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    if audioparams.usemfcc
        # mfcc module
        ncoeffs = audioparams.ncoeffs
        rectification = :log                    # :log, :cubic_root
        dither = true

        mfcc = get_mfcc(
            source=melspec,
            ncoeffs=ncoeffs,
            rectification=rectification,
            dither=dither,
        );
    end

    if audioparams.usef0
        # f0 module
        method = :nfc
        f0_range = (50, 400)

        f0 = get_f0(
            source=stftspec,
            method=method,
            freq_range=f0_range
        );
    end

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        filter(!isnothing, [
            melspec.spec',
            audioparams.usemfcc ? mfcc.mfcc' : nothing,
            audioparams.usef0 ? f0.f0 : nothing,
            spect.centroid,
            spect.crest,
            spect.entropy,
            spect.flatness,
            spect.flux,
            spect.kurtosis,
            spect.rolloff,
            spect.skewness,
            spect.decrease,
            spect.slope,
            spect.spread
        ])...
    )    
end

afe (generic function with 2 methods)

### Prepare dataset for analysis

In [31]:
d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

memguard && begin
    cat2 = round(Int, length(y)/2)
    indices = [1:n_elems; cat2:cat2+n_elems-1]
    x = x[indices, :]
    y = y[indices]
end

freq = round.(Int, afe(x[1, :audio], audioparams; get_only_melfreq=true))

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.nbands],
        audioparams.usemfcc ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.ncoeffs] : String[],
        audioparams.usef0 ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)
    
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

if !loadset
    @info("Build dataset...")

    X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
    audiofeats = [afe(row[:audio], audioparams) for row in eachrow(x)]
    push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audiofeats])...)

    yc = CategoricalArray(y);

    train_ratio = 0.8

    train, test = partition(eachindex(yc), train_ratio, shuffle=true)
    X_train, y_train = X[train, :], yc[train]
    X_test, y_test = X[test, :], yc[test]

    println("Training set size: ", size(X_train), " - ", length(y_train))
    println("Test set size: ", size(X_test), " - ", length(y_test))
end

┌ Info: Build dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:42


Training set size: (403, 225) - 403
Test set size: (101, 225) - 101


### Train a model

In [32]:
if !loadset
    learned_dt_tree = begin
        Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
        model = Tree(max_depth=-1, )
        mach = machine(model, X_train, y_train)
        fit!(mach)
        fitted_params(mach).tree
    end
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


max(cntrd) < 1175.0
├─ min(mel8=1194Hz) < -4.23
│  ├─ std(mel13=2832Hz) < 0.1287
│  │  ├─ std(mel9=1419Hz) < 0.2263
│  │  │  ├─ Pneumonia (103/103)
│  │  │  └─ med(cntrd) < 695.1
│  │  │     ├─ min(crest) < 31.03
│  │  │     │  ⋮
│  │  │     │  
│  │  │     └─ qnt(skwns) < 2.01
│  │  │        ⋮
│  │  │        
│  │  └─ min(mel14=3366Hz) < -7.396
│  │     ├─ Healthy (12/12)
│  │     └─ min(kurts) < 5.44
│  │        ├─ med(mel6=845Hz) < -4.571
│  │        │  ⋮
│  │        │  
│  │        └─ 3ac(mel13=2832Hz) < 0.005859
│  │           ⋮
│  │           
│  └─ min(sprd) < 318.6
│     ├─ Healthy (22/22)
│     └─ Pneumonia (3/3)
└─ std(flatn) < 0.03835
   ├─ mean(mel9=1419Hz) < -4.414
   │  ├─ Pneumonia (8/8)
   │  └─ qnt(mel9=1419Hz) < 1.76
   │     ├─ Pneumonia (1/1)
   │     └─ Healthy (8/8)
   └─ med(sprd) < 204.4
      ├─ Pneumonia (2/2)
      └─ bsm(mel8=1194Hz) < 3.5
         ├─ 3ac(crest) < 0.01251
         │  ├─ Healthy (1/1)
         │  └─ Pneumonia (2/2)
         └─ bsm(mel14=3366H

### Model inspection & rule study

In [33]:
if !loadset
    sole_dt = solemodel(learned_dt_tree)
    # Make test instances flow into the model, so that test metrics can, then, be computed.
    apply!(sole_dt, X_test, y_test);
    # Save solemodel to disk
    jldsave(jld2file, true; sole_dt)
else
    @info("Load dataset...")
    d = jldopen(jld2file)
    sole_dt = d["sole_dt"]
    close(d)
end
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ max(cntrd) < 1175.142326739371
├✔ min(mel8=1194Hz) < -4.230496791684855
│├✔ std(mel13=2832Hz) < 0.12871913235095664
││├✔ std(mel9=1419Hz) < 0.22633671315471646
│││├✔ Pneumonia : (ninstances = 27, ncovered = 27, confidence = 1.0, lift = 1.0)
│││└✘ med(cntrd) < 695.0784158201961
│││ ├✔ min(crest) < 31.03081642487632
│││ │├✔ Pneumonia : (ninstances = 4, ncovered = 4, confidence = 1.0, lift = 1.0)
│││ │└✘ mean(decrs) < -2.316178109209957
│││ │ ├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││ │ └✘ Healthy : (ninstances = 1, ncovered = 1, confidence = 0.0, lift = NaN)
│││ └✘ qnt(skwns) < 2.0096343428350045
│││  ├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││  └✘ Healthy : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
││└✘ min(mel14=3366Hz) < -7.396015851085238
││ ├✔ Healthy : (ninstances = 6, ncovered = 6, confidence = 1.0, lift = 1.0)
││ └✘ min(kurts) < 5.440264768459491
││  ├✔ med(mel6=845Hz) < -4.5711042924882

### Extract rules that are at least as good as a random baseline model

In [34]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.230496791684855) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (std(mel9=1419Hz) < 0.22633671315471646)  ↣  Pneumonia : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 1.0, lift = 2.2, natoms = 4)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.230496791684855) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (¬(std(mel9=1419Hz) < 0.22633671315471646)) ∧ (med(cntrd) < 695.0784158201961) ∧ (min(crest) < 31.03081642487632)  ↣  Pneumonia : (ninstances = 101, ncovered = 4, coverage = 0.04, confidence = 1.0, lift = 2.2, natoms = 6)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.230496791684855) ∧ (std(mel13=2832Hz) < 0.12871913235095664) ∧ (¬(std(mel9=1419Hz) < 0.22633671315471646)) ∧ (¬(med(cntrd) < 695.0784158201961)) ∧ (¬(qnt(skwns) < 2.0096343428350045))  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.84, natoms = 6)
▣ (max(cntrd) < 1175.1423267393

### Simplify rules while extracting and prettify result

In [35]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.23) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) < 0.23)  ↣  Pneumonia : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 1.0, lift = 2.2, natoms = 4)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.23) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) ≥ 0.23) ∧ (med(cntrd) < 695.08) ∧ (min(crest) < 31.03)  ↣  Pneumonia : (ninstances = 101, ncovered = 4, coverage = 0.04, confidence = 1.0, lift = 2.2, natoms = 6)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.23) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) ≥ 0.23) ∧ (med(cntrd) ≥ 695.08) ∧ (qnt(skwns) ≥ 2.01)  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.84, natoms = 6)
▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.23) ∧ (std(mel13=2832Hz) ≥ 0.13) ∧ (min(mel14=3366Hz) < -7.4)  ↣  Healthy : (ninstances = 101, ncovered = 6, coverage = 0.06, confidence = 1.0, lift = 1.84, natoms = 4)
▣ (max(cntrd) < 1175.14) ∧ (min

### Directly access rule metrics

In [36]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

12-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 27, coverage = 0.26732673267326734, confidence = 1.0, lift = 2.1956521739130435, natoms = 4)
 (ninstances = 101, ncovered = 4, coverage = 0.039603960396039604, confidence = 1.0, lift = 2.1956521739130435, natoms = 6)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 1.0, lift = 1.8363636363636362, natoms = 6)
 (ninstances = 101, ncovered = 6, coverage = 0.0594059405940594, confidence = 1.0, lift = 1.8363636363636362, natoms = 4)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8363636363636362, natoms = 7)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.1956521739130435, natoms = 7)
 (ninstances = 101, ncovered = 4, coverage = 0.039603960396039604, confidence = 0.75, lift = 1.6467391304347825, natoms = 

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [37]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.230496791684855) ∧ (std(mel13=2832Hz) ≥ 0.12871913235095664) ∧ (min(mel14=3366Hz) ≥ -7.396015851085238) ∧ (min(kurts) < 5.440264768459491) ∧ (med(mel6=845Hz) < -4.571104292488293) ∧ (max(mel3=504Hz) < -3.2631597021121945)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8363636363636362, natoms = 7, height = 6)
▣ (max(cntrd) < 1175.142326739371) ∧ (min(mel8=1194Hz) < -4.230496791684855) ∧ (std(mel13=2832Hz) ≥ 0.12871913235095664) ∧ (min(mel14=3366Hz) ≥ -7.396015851085238) ∧ (min(kurts) < 5.440264768459491) ∧ (med(mel6=845Hz) ≥ -4.571104292488293) ∧ (max(kurts) < 12.927201651607955)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.1956521739130435, natoms = 7, height = 6)
▣ (max(cntrd) ≥ 1175.142326739371) ∧ (std(flatn) < 0.038350928797190896) ∧ (mean(mel9=1419Hz) < -4.413854005770021)  ↣  Pneumonia : (ninstance

### Pretty table of rules and their metrics

In [38]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                              Antecedent │ Consequent │ ninstances │ ncovered │   coverage │ confidence │    lift │ natoms │ height │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Inspect features

In [39]:
interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
);
map(r->(consequent(r), readmetrics(r)), interesting_rules)
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ (max(cntrd) < 1175.14) ∧ (min(mel8=1194Hz) < -4.23) ∧ (std(mel13=2832Hz) < 0.13) ∧ (std(mel9=1419Hz) < 0.23)  ↣  Pneumonia : (ninstances = 101, ncovered = 27, coverage = 0.27, confidence = 1.0, lift = 2.2, natoms = 4)
▣ (max(cntrd) ≥ 1175.14) ∧ (std(flatn) ≥ 0.04) ∧ (med(sprd) ≥ 204.43) ∧ (bsm(mel8=1194Hz) ≥ 3.5) ∧ (bsm(mel14=3366Hz) ≥ 3.5)  ↣  Healthy : (ninstances = 101, ncovered = 29, coverage = 0.29, confidence = 0.86, lift = 1.58, natoms = 5)


In [40]:
interesting_features = unique(SoleData.feature.(SoleLogics.value.(vcat(SoleLogics.atoms.(i.antecedent for i in interesting_rules)...))))
interesting_variables = sort(SoleData.i_variable.(interesting_features))

8-element Vector{Symbol}:
 Symbol("bsm(mel14=3366Hz)")
 Symbol("bsm(mel8=1194Hz)")
 Symbol("max(cntrd)")
 Symbol("med(sprd)")
 Symbol("min(mel8=1194Hz)")
 Symbol("std(flatn)")
 Symbol("std(mel13=2832Hz)")
 Symbol("std(mel9=1419Hz)")